# Série 1: Formalisation
## Les trois médecins

### Description
Alice, Bob et Charles sont trois patients suivis par les médecins Xavier,
Yolande et Zoé. Aujourd’hui, ces 6 personnes doivent être transférées de
l’Hôpital Guéritou vers l’Hôpital Soignetou grâce à une ambulance proposant
deux places. Les médecins ne doivent jamais se retrouver en infériorité
numérique pour prodiguer leur soins. On peut toutefois avoir des patients
dans un hôpital sans médecins. De plus l’ambulance ne fait pas de trajet à
vide.
Comment va-t-il falloir organiser les diférents trajets pour transporter
les 6 personnes d’un hôpital à l’autre, en respectant les contraintes ?

### Formalisation du problème recherche
Formalisez le problème en extrayant l’information suivante pour représenter
les différentes situations possibles. Profitez de cet exercice pour reviser les
notions du cours.

1. Donnez une représentation des états.

On pourrait représenter un état du système par un tuple de la forme (x, y, z) avec x, le nombre de médecins dans l'Hôpital Guéritou,
y le nombre de patients dans l'Hôpital Guéritou, et z la position de l'ambulance... Pour connaître le nombre de patients et de médecins dans
l'Hôpital Soignetou, il suffit de faire respectivement le nombre total de médecins moins le nombre de médecins dans l'Hôpital Guéritou et le nombre total
de patients moins le nombre de patients dans l'Hôpital Soignetou.

L'ambulance est soit à l'Hôpital Guéritou, soit à l'Hôpital Soignetou. Pour modéliser le problème, on ne va donc pas considérer le temps de trajet car cela n'apporterait aucune valeur
ajoutée...

Dans notre cas, on a 3 médecins et 3 patients. L'espace des états possibles du système est donc donné par $S = (x, y, z)$ avec $x, y \in {0, 1, 2, 3}$ car il y a 3 médecins
et 3 patients, $x \geq y$ ou $x = 0$ car les médecins ne doivent jamais être en infériorité numérique pour prodiguer leurs soins et qu'on peut avoir des patients dans un
hôpital sans médecins, et $z \in {0, 1}$ car l'ambulance est soit à l'Hôpital Guéritou, soit dans l'Hôpital Soignetou. Comme notre référentiel est l'Hôpital Guéritou, l'ambulance est
soit là, soit pas là.

On peut définir l'espace d'états par: W = ({0-3}, {0-3}, {0-1})

2. Quels sont les opérateurs de transition possibles ?

Les opérateurs de transition possibles sont représentés par la fonction de transition:
$$\Gamma : S \rightarrow S$$
La fonction de transition est la contrainte de l'ambulance: l'ambulance a 2 places et ne fait jamais de trajets à vide.

3. Définissez les conditions pour lesquelles les opérateurs sont applicables.

Les opérateurs sont applicables si on peut passer d'un état appartenant à l'espace d'états du système à un autre espace appartenant à l'espace d'états du système.
Donc les opérateurs sont applicables si:
$ f (x, y, z) = (x', y', z')$ avec $x', y' \in {0, 1, 2, 3}$, et ($x' \geq y'$ ou $x'=0$) et si z' = 1, alors $x'+y' - (x+y) \in {1, 2}$ et si z'=0, alors $x+y - (x'+y') \in {1, 2}$

4. En vous basant sur le _TP0_, implementez un algorithme de recherche
pour résoudre le problème en utilisant un arbre de recherche correspondant à la description que vous avez choisi.

In [13]:
import numpy as np

def list_possibilities(node: list[int]) -> list[list[int]]:
    k = 0
    if node[2] == 0:
        k = 1
    result = []
    for i in np.arange(4):
        for j in np.arange(4):
            if ispossible(node, [i, j, k]):
                result.append([i, j, k])
    return result
                

# Node: doctors, cobayes, cars

def ispossible(node: list[int], neighbourg: list[int]) -> bool:
    if node[2] != neighbourg[2]:
        var = 0
        if (neighbourg[0] >= neighbourg[1] or neighbourg[0] == 0):
            if node[2] == 0:
                var = neighbourg[0] + neighbourg[1] - (node[0] + node[1])
            else:
                var = node[0] + node[1] - (neighbourg[0] + neighbourg[1])
        if (var == 1 or var == 2):
            return True
    return False

def stop_condition(node: list[int]) -> bool:
    if (node[0] != 0 or node[1] != 0):
        return False
    return True

def shortest_path_len_graph(start: list[int]) -> int:
    tuples = []
    tuples.append(start)
    visited = []
    tovisit = []
    path_length = 1
    while (len(tuples) != 0):
        node = tuples.pop(0)
        possibilities = list_possibilities(node)
        for i in possibilities:
            if i not in visited:
                visited.append(i)
                if stop_condition(node):
                    print("Success")
                    return path_length
                tovisit.append(i)
        if (len(tuples) == 0 and len(tovisit) != 0):
            tuples = tovisit.copy()
            tovisit = []
            path_length += 1
    print("Failure")
    return -1
shortest_path_len_graph([3, 3, 1])

Success


10

### Complexité

1. Calculez la taille de l’espace de recherche. Pour taille de l’espace de
recherche on considère tout l’espace de recherche, y compris les états
où les patients ne sont plus accompagnés.

La taille de l'espace de recherche est donné par:

Taille de S: 
$$4 * 4 * 2 - \sum_{i=1}^{3}\sum_{j=i+1}^{4}1 - 1 - 1
= 32 - \sum_{i=0}^{2}(\sum_{j=i}^{3} 1)
= 32 - (3+2+1) - 1 - 1
= 24$$

Les deux -1 sont causés par la contrainte de l'ambulance: on ne peut pas avoir 0 patients et médecins dans un hôpital avec l'ambulance à cet hôpital car elle ne fait jamais
de trajets à vide et si on a 3 patients et médecins dans un hôpital, on ne peut pas avoir l'ambulance qui est à l'autre hôpital pour la même raison.

In [3]:
def size_search_space(nodes: list[list[int]], current: list[int]) -> list[list[int]]:
    nodes.append(current)
    possibilities = list_possibilities(current)
    for i in possibilities:
        if i not in nodes:
            size_search_space(nodes, i)
    return nodes
print(len(size_search_space([], [3, 3, 1])))

24


2. Calculez le nombre d’états où chaque patient est accompagné.

Le nombre d'états où chaque patient est accompagné est égal à:

Nombre d'états de S - possibilité qu'il y ait 0 médecins dans l'hôpital Guéritou alors que l'ambulance est là ($\sum_{i=1}^{4} 1 = 4$) plus la possibilité qu'il y ait des patients pas
accompagnés alors que l'ambulance n'est pas là ($\sum_{i=1}^{4} 1 = 4$)

In [4]:
def patients() -> int:
    tuples = size_search_space([], [3, 3, 1])
    n = len(tuples)
    for j in np.arange(2):
        for i in np.arange(1, 4):
            if [0, i, j] in tuples:
                n -= 1
        for i in np.arange(3):
            if [3, i, j] in tuples:
                n -= 1
    return n

print(patients())
        

12


3. Calculez le nombre d’états accessibles depuis l’état initial.

Le nombre d'états accessibles depuis l'état initial est, comme nous l'avons calculé précédement avec notre fonction, de 24.

## Les tours de Hanoi

### Description

Les tours de Hanoi est un casse-tête dont le but est de transférer une tour
d’un poteau à l’autre. Le jeu se compose de n disques de tailles croissante
enilés sur 3 poteaux. Au début du jeu, les n disques forment une tour
croissante sur le poteau de gauche. Le but est de transférer cette tour sur
le poteau de droite en utilisant aussi le poteau du milieu. On ne peut pas
empiler un disque sur un autre plus petit.

### Questions
1. Formalisez le problème:
   - formalisation d'un état
   - formalisation de l'état initial et de l'état final
   - formalisation des transitions

On pourrait représenter un état par un tuple (x, y, z) avec x, y, z la taille du disque du sommet de la pile de disque, tel que $x, y, z \in {0-n}$. 0 représente l'état où la tour
i ne possède aucun disque... L'espace des états possibles est donc donné par $S = n * (n - 1) * (n - 2)$

L'état initial est donné par (1, 0, 0) avec 1 le disque le plus petit en taille et n le disque le plus grand en taille

L'état final est donné par (0, 0, 1)

On peut définir la fonction de transition comme:
$f: S \rightarrow S$
tel que f(tuple_1) = tuple_2 avec:

Pour $i \neq j \neq k$, si tuple_1[i] == tuple_2[j], alors $tuple_1[i] \gt tuple_2[i]$, $tuple_1[j] \lt tuple_2[j]$, $tuple_1[k] \eq tuple_2[k]$

2. Combien y a-t-il d'états possibles pour $n \eq 3$ ?

Pour n = 3, on a:

3. En vous basant sur le TP0, implémentez la solution pour $n=3$

In [12]:
def transition_function(start: list[int], goal: list[int]) -> bool:
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if (i != j != k):
                    if (start[i] == goal[j] and (start[i] < goal[i] or goal[i] == 0) and (start[j] > goal[j] or start[j] == 0) and start[k] == goal[k]): return True
    return False

def list_possibilities(start: list[int]) -> list[list[int]]:
    result = []
    for i in range(4):
        for j in range(4):
            for k in range(4):
                if transition_function(start, [i, j, k]):
                    result.append([i, j, k])
    return result

def DLS(start: list[int], goal: list[int], path: list[list[int]], limit: int) -> bool:
    if start == goal: return True
    if limit <= 0: return False
    possibilities = list_possibilities(start)
    for i in possibilities:
        if DLS (i, goal, path, limit) == True:
            path.append(i)
            return True
    return False

def IDDFS(start: list[int], goal: list[int], limit: int) -> list[list[int]]:
    for i in range(limit):
        path = []
        if DLS(start, goal, path, i): return path
    return []

print(IDDFS([0, 0, 1], [1, 0, 0], 20))
                    

RecursionError: maximum recursion depth exceeded in comparison